In [1]:
from platform import python_version
print(python_version())

3.9.7


## Collecting the data

In [2]:
!pip install -q -U watermark

In [3]:
!pip install -q -U pandas_datareader

In [4]:
# US holidays
!pip install -q -U holidays

In [5]:
# Imports
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
import time
# Dataframes
import pandas as pd
import numpy as np
# To connect to the URL
import requests
import holidays
import bs4
import pandas_datareader
from random import randint
from datetime import date
# Web Scrap.
from bs4 import BeautifulSoup
# To know the name of the companies
from pandas_datareader import data as pdr

### Yahoo Finance

In [6]:
# ticker_list: code list of each company 
# list with the company's cotes
# ticker : code of the company
# list_of_df: list of codes
def return_stock(ticker_list, start, end):
    list_of_df = []
    not_found = []
    
    for ticker in ticker_list:
        try:
            df = pdr.get_data_yahoo(ticker, start, end)
            df = df.reset_index()
            df.rename(columns = {'Date':'date'})
            df['ticker'] = [ticker] * len(df)
            list_of_df.append(df)
        except ValueError:
            print("Not found: " + ticker)
            not_found.append(ticker)
    
    return list_of_df, not_found

### NASDAQ

In [7]:
def scrape_nasdaq(ticker_list):
    tckr_data = {}
    count = 1
    
    for ticker in ticker_list:
        print("Scraping count: " + str(count))
        
        url = 'https://old.nasdaq.com/symbol/' + ticker + '/historical'
        driver.get(url)
        
        # Collect data from 10 years
        # ddlTimeFrame tag html
        data_range = driver.find_elements_by_name('ddlTimeFrame')
        
        if len(data_range) > 0: # =0 if do not find ddlTimeFrame
            for option in data_range[0].find_elements_by_tag_name('option'): # option: time frame in url
                if option.text == '10 Years':
                    option.click()
                    break
            time.sleep(5) # wait until page load the data
            page_source = driver.page_source # save the data in page_source
    
            # Extraction data
            soup = BeautifulSoup(page_source, 'lxml')
    
            # Defining the search rule in the data table
            tags = soup.find_all('div', id = "historicalContainer")
    
            # Put the data in :
            temp_data = []
            # Loop:
            for tag in tags:
                rows = tag.findAll('tr')
                for tr in rows:
                    cols = tr.findAll('td')
                    val = [tr.text for tr in cols]
                    temp_data.append(val)
            
            # Cleaning te data
            for i in range(len(temp_data)):
                to_process = temp_data[i]
    
                for i in range(len(to_process)):
                    temp = to_process[i]
                    temp = temp.strip()
                    to_process[i] = temp
         
            # Getting data
            temp_data = temp_data[2:]
            # Saving temporary data
            if tckr_data.get(ticker) == None:
                tckr_data[ticker] = temp_data # dictionary
        
            # Pause to avoid be blocked 
            print("Random Sleep")
            sleep(randint(2, 4))
            count += 1
        
        else:
            print("Not found" + ticker)
        
    # Prepare lists
    date = []
    Open = []
    high = []
    low = []
    close = []
    volume = []
    ticker = []

    # Collect data & append
    for key, value in tckr_data.items():
        for data in value:
            date.append(data[0].replace('/','-'))
            Open.append(data[1])
            high.append(data[2])
            low.append(data[3])
            close.append(data[4])
            volume.append(data[5].replace(',',''))
            ticker.append(key)  
            
    # Final dictionary 
    final_dict = {'date' : date, 'Open':Open, 'High':high, 'Low':low, 'Close':close, 'Volume':volume, 'ticker':ticker}
    
    # Convert to dataframe
    df = pd.DataFrame(final_dict)
    cols = ['date', 'Open', 'High', 'Low', 'Close', 'Volume', 'ticker']
    df = df[cols]

    return df

### All together

In [8]:
def get_stocks_data(ticker_set, start, end):
    
    # Yahoo Finance: stocks_1 
    stocks_1, not_found = return_stock(ticker_set, start, end)
    # If <=0 stocks were found in Yahoo Finance, if not:
    if len(not_found) > 0:
    
        # Collect from Nasdaq
        stocks = scrape_nasdaq(not_found)
    
        # Merging in dataframe
        stocks_1 = pd.concat(stocks_1)

        stocks_1 = stocks_1.rename(columns = {'Date' : 'date'})
        stocks_1 = stocks_1[['date', 'Open', 'High', 'Low', 'Close', 'Volume', 'ticker']]
        stocks_1['date'] = stocks_1['date'].dt.date

        stocks = stocks.rename(columns = {'Ticker' : 'ticker'})
        stocks[['Open', 'High', 'Low', 'Close', 'Volume']] = stocks[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)
    
        stocks_final = pd.concat([stocks_1, stocks])
        return stocks_final
    
    else:
        return pd.concat(stocks_1)

In [9]:
# Stock code list
tickers = ['AAPL', 'XOM' ,'VMC', 'BA', 'AMZN', 'TGT', 'WMT', 'KO', 'UNH', 'JPM', 'GOOGL', 'STT', 'MSFT', 'VZ', 'XEL', 'SPG']

In [10]:
# Colleting price from 10 years
df = get_stocks_data(tickers, start = '2009-01-01', end = '2019-12-31')

In [11]:
df.sample(5)

,Date,High,Low,Open,Close,Volume,Adj Close,ticker
2380,2018-06-18,77.919998,76.190002,76.330002,77.120003,3341000.0,69.638611,TGT
1171,2013-08-28,67.290001,66.470001,66.750000,66.790001,1670500.0,53.592091,STT
1670,2015-08-21,93.050003,90.800003,92.500000,91.660004,1316700.0,85.993980,VMC
1676,2015-08-31,33.860001,33.220001,33.820000,33.730000,5646200.0,27.147259,XEL
1714,2015-10-23,37.200001,36.310001,37.070000,36.439999,3624600.0,29.614157,XEL


In [12]:
# Removing Volume and Adj Close from the table
df = df[['Date', 'Open', 'High', 'Low', 'Close', 'ticker']]

In [13]:
df.sample(5)

,Date,Open,High,Low,Close,ticker
428,2010-09-15,88.410156,90.188148,88.165565,89.764816,SPG
72,2009-04-17,4.327857,4.437500,4.294643,4.407857,AAPL
2171,2017-08-17,46.119999,46.330002,45.849998,45.860001,KO
1169,2013-08-26,64.529999,64.900002,64.099998,64.129997,TGT
387,2010-07-19,49.720001,49.810001,49.090000,49.520000,WMT


In [15]:
# csv
df.to_csv('dados/stocks.csv')